In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale

letters = pd.read_csv("letter-recognition.csv")

In [ ]:
print("Dimensions: ", letters.shape, "\n")
# print(letters.info())
# letters.head()

Dimensions:  (20000, 17) 



In [ ]:
letters.columns = ['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey', 'yedge', 'yedgex']
# print(letters.columns)

In [ ]:
letters = letters[letters['letter'].isin(['A', 'B']) == True] 
# letters = letters[(letters['letter'].any('A', 'B'))] 

In [ ]:
order = list(np.sort(letters['letter'].unique()))
print(order)

['A', 'B']


In [ ]:
letter_means = letters.groupby('letter').mean()
letter_means.head()

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
letter,,,,,,,,,,,,,,,,
A,3.337136,6.975919,5.128010,5.178707,2.991128,8.851711,3.631179,2.755387,2.043093,7.802281,2.338403,8.465146,2.771863,6.321926,2.875792,7.468948
B,3.985640,6.962141,5.088773,5.169713,4.596606,7.671018,7.062663,5.366841,5.571802,7.954308,5.506527,6.652742,3.117493,7.919060,6.612272,9.100522


In [ ]:
round(letters.drop('letter', axis=1).mean(), 2)

xbox      3.66
ybox      6.97
width     5.11
height    5.17
onpix     3.78
xbar      8.27
ybar      5.32
x2bar     4.04
y2bar     3.78
xybar     7.88
x2ybar    3.90
xy2bar    7.57
xedge     2.94
xedgey    7.11
yedge     4.72
yedgex    8.27
dtype: float64

In [ ]:
X = letters.drop("letter", axis = 1)
y = letters['letter']

In [ ]:
X_scaled = scale(X, with_mean=False)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 101)

In [ ]:
import random
print("Number of training samples = ", len(X_train))
print("Number of testing samples = ", len(X_test))

Number of training samples =  1088
Number of testing samples =  467


In [ ]:
# Linear SVM model

model_linear = SVC(kernel='linear')
model_linear.fit(X_train, y_train)
y_pred = model_linear.predict(X_test)

In [ ]:
print("Accuracy linear kernel without Markov Sampling : ", metrics.accuracy_score(y_true=y_test, y_pred=y_pred), "\n")
print(metrics.confusion_matrix(y_true=y_test, y_pred=y_pred))

Accuracy linear kernel without Markov Sampling :  0.9914346895074947 

[[225   1]
 [  3 238]]


In [ ]:
import math
def markov_samp(X_train, Y_train, k = 5, q = 1.2, n2 = 10, T = 10):

#Taking N1 as 1000
    # Dtrain = np.concatenate((X_train, np.array()), axis = 0)
    Dtrain = np.concatenate((np.vstack(X_train), np.vstack(Y_train.to_numpy())), axis = 1)
    Do = random.sample(list(Dtrain), 1000)

    N = len(Do)
    print("N : " , N)

#Initializing mneg and mpos as 0
    t = 1

#Choosing first zt random sample from chosen training samples
    index = np.random.choice(len(Do), 1, replace=False)  
    zsample = np.array(Do)
    # print(type(zsample))
    z = zsample[index][0]

    # print("z : ", z)

#Initializing empty sample holder
    samp = []

#Training SVM for choosing Markov CHain Samples
    model_linear = SVC(kernel='linear')#
    model_linear.fit(X_train, Y_train)#
    
    yzt = model_linear.predict(np.array(z)[0:16].reshape(1, -1))
#Calculating loss for zt sample
    fxy1 = 1
    if not yzt==z[16]:
        fxy1 = 2 
    lzt = math.exp(0-fxy1)

    Pd = 0
    Pdd = 0
    zi = z
#Loop for Markov sampling    
    while(t <= T):
        i = 1
        n1 = 0
        samp = []
        # print(str(t) + " samp " + str(len(samp)) + " N "+ str(N))
        while(i <= N):
            # print("i ", i)
        #Choosing zstar sample 
            zstar = Dtrain[np.random.choice(len(Dtrain), 1, replace=False)][0]
            ystar = model_linear.predict(np.array(zstar)[ 0:16].reshape(1, -1)) #
        #Calculating loss for zstar and P ratio
            fxy = 1
            if not ystar==zstar[16]:
                fxy = 2 
            lzstar = math.exp(0-fxy)

            yzt = model_linear.predict(np.array(zi)[0:16].reshape(1, -1))
            #Calculating loss for zi sample
            fxy1 = 1
            if not yzt==zi[16]:
                fxy1 = 2 
            lzt = math.exp(0-fxy1)

            P = min(1.0, math.exp(lzt-lzstar))

            # print(zt)
            yt = zi[16]
            zi = zstar

            if n1 > n2:
                P = min(1.0, q*P)
                zi = zstar
                Dt = zi
                i = i+1
                n1 = 0

        #Accept/Reject
            rejected = 0
            if P == 1:
                if zi[16] == yt:
                    r = random.uniform(0.001, 1.0)
                    if r <= P:
                        samp.append(zstar) #Pdash = e−y∗f0 /e−ytf0
                        i += 1
                    else:
                        rejected = 1
                else:
                    ct = 1 if yt=='A' else -1
                    cst = 1 if ystar=='A' else -1
                    Pd = math.exp(ct*fxy1-cst*fxy)
                    r = random.uniform(0.001, 1.0)
                    if r <= Pd:
                        samp.append(zstar) #P
                        i += 1
                    else:
                        rejected = 1

        #Updating zi          
            zi = zstar
            
        #Checking accept criteria final time and updating n1 if rejection
            if P > 1 or Pd > 1 or Pdd > 1:
                samp.append(zstar)
                i += 1
            if rejected == 1:
                n1 = n1 + 1
            
            # print(len(samp))    
        zi = zstar
        t = t+1

        dtx = np.array(samp)[:, 0:16]
        dty = np.array(samp)[:, 16]
        dtY = []
        for i in dty:
            dtY.append(ord(i))

        model_linear = SVC(kernel='linear')#
        model_linear.fit(dtx, dty)#
        
        decision_function = model_linear.decision_function(dtx)
        support_vector_indices = np.where(np.abs(decision_function) <= 1 + 1e-15)[0]
        support_vectors = dtx[support_vector_indices]
        print("Support vectors for Dt -> ")
        print(support_vectors)

        y_pred = model_linear.predict(dtx)
        et = 1.0 - metrics.accuracy_score(y_true=dty, y_pred=y_pred)
        if et != 0:
            alpha = 0.5*math.log((1-et)/et)
            if alpha < 0:
                t = t-1

    return samp


In [ ]:
nsamp = np.array(markov_samp(X_train, y_train))

X_train = nsamp[:, 0:16]
y_train = nsamp[:, 16]
print("Shape of Markov Sample features : ", X_train.shape)
print("Shape of Markov Sample labels : ", y_train.shape)

N :  1000
Support vectors for Dt -> 
[['2.9247342868016686' '3.2633061255196103' '4.032974345546051'
  '3.6456830157304077' '3.242956848511186' '5.195284116875042'
  '2.6709041525121298' '1.367267001444668' '3.3021544317717724'
  '5.880005078134125' '1.8434678944984777' '5.7418469666929495'
  '1.8546149678515584' '5.19744570255692' '2.0157012483142696'
  '6.194548585975666']
 ['2.9247342868016686' '3.2633061255196103' '4.032974345546051'
  '3.6456830157304077' '3.242956848511186' '5.195284116875042'
  '2.6709041525121298' '1.367267001444668' '3.3021544317717724'
  '5.880005078134125' '1.8434678944984777' '5.7418469666929495'
  '1.8546149678515584' '5.19744570255692' '2.0157012483142696'
  '6.194548585975666']
 ['2.3397874294413348' '2.3733135458324437' '3.456835153325186'
  '3.1899726387641065' '2.779677298723874' '4.040776535347255'
  '3.5612055366828397' '0.9115113342964453' '1.88694538958387'
  '4.116003554693887' '3.6869357889969554' '5.7418469666929495'
  '3.0910249464192643' '5.9

In [ ]:
# print(y_train)
#Converting label to ASCII in list for sklearn as sampling returned ndarray
Y_train = []
for i in y_train:
    Y_train.append(ord(i))
# print(Y_train)

In [ ]:
print("", X_test.shape)

 (467, 16)


In [ ]:
# print(y_pred)
#Converting label to ASCII in list for sklearn as training is done on ASCII
Y_test = []
for i in y_test:
    Y_test.append(ord(i))
# print(Y_test)

In [ ]:
# Linear SVM model

model_linear = SVC(kernel='linear')
model_linear.fit(X_train, Y_train)
y_pred = model_linear.predict(X_test)


In [ ]:
# accuracy
print("Accuracy Linear kernel with Improved Markov Boosting : ", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")
print(metrics.confusion_matrix(y_true=Y_test, y_pred=y_pred))

Accuracy Linear kernel with Improved Markov Boosting :  0.9957173447537473 

[[225   1]
 [  1 240]]
